In [1]:
# !pip install -r requirements.txt

In [2]:
import asyncio
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
import os
from pathlib import Path
from flask import Flask

# Define the path to your MCP server
server_params = StdioServerParameters(
    command="docker",
    args=[
        "run",
        "-i",
        "--rm",
        "-e",
        "GITHUB_PERSONAL_ACCESS_TOKEN",
        "ghcr.io/github/github-mcp-server"
      ],
    env={
        "GITHUB_PERSONAL_ACCESS_TOKEN": os.environ.get("GITHUB_PERSONAL_ACCESS_TOKEN"),
    }
)

async def ai(query: str):

    data = ""
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            tools = await load_mcp_tools(session)
            llm = ChatOpenAI(model="gpt-4.1", temperature=0)

            agent_executor = create_react_agent(llm, tools)

            # System prompt allows for broad range of instructions. But what does user input look like?
            events = agent_executor.astream(
                {"messages": [("system",
                               "You are a web server, serving HTML web pages. Do not use markdown for formatting. "
                               "All output must be in HTML format and will be displayed to an end user. "
                               "You need to start with doctype and html tags and provide the entire page"),
                              ("user", query)]},
                stream_mode="values",
            )

            async for event in events:
                event["messages"][-1].pretty_print()
                data = event["messages"][-1]

    return data.content


        

app = Flask(__name__)
    
@app.route('/', methods=['GET'])
def get_data():
    query = Path("my_app-step-3-mcp.txt").read_text()
    response = asyncio.run(ai(query))
    return response

app.run(debug=False, port=5001, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit


================================ Human Message =================================

Given my repo: https://github.com/microsoft/AI_Agents_Hackathon
Find the most prolific author of github issues, and I want to see all of their issues and comments

I will need to see ONLY the author, title, and comments. HTML elements can be styled using bootstrap css.
================================== Ai Message ==================================
Tool Calls:
  list_issues (call_bLYE36EVzvnkq5rZrnVVvGKS)
 Call ID: call_bLYE36EVzvnkq5rZrnVVvGKS
  Args:
    owner: microsoft
    repo: AI_Agents_Hackathon
    perPage: 100
    state: all
================================= Tool Message =================================
Name: list_issues

[{"id":3059183047,"number":760,"state":"open","locked":false,"title":"Project: Astra - Your AI Companion, Language: Python, Framework: Semantic Kernel, Azure AI Agent Service [Resubmission as previous account suspended/hacked]","body":"### Project Name\n\nAstra - Your AI Compan

127.0.0.1 - - [14/May/2025 22:11:21] "GET / HTTP/1.1" 200 -


================================== Ai Message ==================================

<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>Most Prolific Issue Author - AI_Agents_Hackathon</title>
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.3/dist/css/bootstrap.min.css" rel="stylesheet">
</head>
<body class="bg-light">
  <div class="container my-5">
    <h1 class="mb-4">Most Prolific Issue Author: <span class="text-primary">Jaykef</span></h1>
    <p class="lead">Below are all issues authored by <strong>Jaykef</strong> in <a href="https://github.com/microsoft/AI_Agents_Hackathon" target="_blank">microsoft/AI_Agents_Hackathon</a>, with their titles and all comments.</p>

    <!-- Issue 753 -->
    <div class="card mb-4 shadow">
      <div class="card-header bg-primary text-white">
        <div class="d-flex align-items-center">
          <img src="https://avatars.githubusercontent.com/u/11355002?v=4" alt="Jaykef" class="rounded-circle me-2" width="32" height=

## Goals
- Write natural language only
- Unstructured data
- Implicit algorithms
- ~~Dynamic tools~~
- ~~Human in the loop built-in~~
- ~~Consistent output~~
- ~~Iterative development~~